In [308]:
import torch
from torchvision import datasets
import random
import torch.nn as nn
import numpy as np

In [329]:
device = 'gpu' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [507]:
def get_sequence(n_timesteps, limit_const=4):
    x = torch.tensor([random.random() for _ in range(n_timesteps)])
    limit = len(x)/limit_const
    y = torch.tensor([0 if i < limit else 1 for i in np.cumsum(x)])
    x = x.reshape(1, 1, n_timesteps)
    y = y.reshape(1, n_timesteps, 1)
    return x, y

In [508]:
get_sequence(10)[0].shape

torch.Size([1, 1, 10])

In [608]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers, batch_first=False, bidirectional=True
        )
        self.fc = nn.Linear(hidden_size * 2 , num_classes)
        self.softmax  = nn.Softmax(dim=1)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2 , 1, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers * 2 , 1, self.hidden_size).to(device)
        print(h0.shape, c0.shape)
        out, (h,c) = self.lstm(x, (h0, c0))
#         out = self.softmax(self.fc(out[:, -1 , :]))        
        return out , (h,c)

In [611]:
num_epochs = 1000
input_size =  10
hidden_size = 5
num_layers = 2
num_classes = 10

In [618]:
model = LSTM(input_size, hidden_size, num_layers, num_classes)
out, (h,c) = model(data)
h.shape, c.shape

torch.Size([4, 1, 5]) torch.Size([4, 1, 5])


(torch.Size([4, 1, 5]), torch.Size([4, 1, 5]))

In [606]:
print(data.shape, out.shape, h.shape, c.shape)

torch.Size([1, 1, 10]) torch.Size([1, 1, 10]) torch.Size([2, 1, 5]) torch.Size([2, 1, 5])


In [537]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [538]:
for epoch in range(num_epochs):
    data, targets = get_sequence(10)
    output = model(data)
    loss = criterion(output, targets.squeeze(1).squeeze(0))
    print(f"{epochs}/{num_epochs} , loss : {loss.items()}")
    loss.backward()
    optimizer.step()


ValueError: Expected input batch_size (1) to match target batch_size (10).

In [617]:
output = model(data)
output

torch.Size([4, 1, 5]) torch.Size([4, 1, 5])


(tensor([[[-0.0543, -0.0107, -0.1132, -0.0724, -0.0563, -0.1805, -0.0650,
            0.0238, -0.1040,  0.0258]]], grad_fn=<CatBackward>),
 (tensor([[[-0.2509,  0.0563,  0.1335, -0.0293, -0.0558]],
  
          [[-0.0579, -0.1436,  0.0180, -0.1418,  0.1006]],
  
          [[-0.0543, -0.0107, -0.1132, -0.0724, -0.0563]],
  
          [[-0.1805, -0.0650,  0.0238, -0.1040,  0.0258]]],
         grad_fn=<StackBackward>),
  tensor([[[-0.3990,  0.1449,  0.3845, -0.0950, -0.1125]],
  
          [[-0.1462, -0.2212,  0.0288, -0.4348,  0.2392]],
  
          [[-0.1288, -0.0305, -0.2144, -0.2075, -0.1032]],
  
          [[-0.3256, -0.1973,  0.0503, -0.2474,  0.0553]]],
         grad_fn=<StackBackward>)))

In [581]:
data.shape

torch.Size([1, 1, 10])

In [582]:
criterion(model(data), targets.reshape(1,10))

IndexError: Target 1 is out of bounds.

In [599]:
criterion(torch.randn(40,10), torch.randint(0,10, (1,40)).reshape(40))

tensor(2.7343)

In [598]:
torch.randint(0,10, (1,40)).shape

torch.Size([1, 40])